# Transfer-learning on EffNetV2

In [ ]:
!pip install split-folders

In [ ]:
pip install -U keras-efficientnet-v2

In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import PIL
import splitfolders
import tensorflow as tf
import numpy as np
import os

In [36]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard, BackupAndRestore
# from tensorflow.keras.callbacks.experimental import BackupAndRestore

In [46]:
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

In [47]:
def plot_images(images, cls_true, cls_pred=None, smooth=True):

    assert len(images) == len(cls_true)

    # Create figure with sub-plots.
    fig, axes = plt.subplots(3, 3)

    # Adjust vertical spacing.
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    # Interpolation type.
    if smooth:
        interpolation = 'spline16'
    else:
        interpolation = 'nearest'

    for i, ax in enumerate(axes.flat):
        # There may be less than 9 images, ensure it doesn't crash.
        if i < len(images):
            # Plot image.
            ax.imshow(images[i],
                      interpolation=interpolation)

            # Name of the true class.
            cls_true_name = class_names[cls_true[i]]

            # Show true and predicted classes.
            if cls_pred is None:
                xlabel = "True: {0}".format(cls_true_name)
            else:
                # Name of the predicted class.
                cls_pred_name = class_names[cls_pred[i]]

                xlabel = "True: {0}\nPred: {1}".format(cls_true_name, cls_pred_name)

            # Show the classes as the label on the x-axis.
            ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [48]:
# Import a function from sklearn to calculate the confusion-matrix.
from sklearn.metrics import confusion_matrix

def print_confusion_matrix(cls_pred):
    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # Get the confusion matrix using sklearn.
    cm = confusion_matrix(y_true=cls_test,  # True class for test-set.
                          y_pred=cls_pred)  # Predicted class.

    print("Confusion matrix:")
    
    # Print the confusion matrix as text.
    print(cm)
    
    # Print the class-names for easy reference.
    for i, class_name in enumerate(class_names):
        print("({0}) {1}".format(i, class_name))

In [49]:
def plot_example_errors(cls_pred):
    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # Boolean array whether the predicted class is incorrect.
    incorrect = (cls_pred != cls_test)

    # Get the file-paths for images that were incorrectly classified.
    image_paths = np.array(image_paths_test)[incorrect]

    # Load the first 9 images.
    images = load_images(image_paths=image_paths[0:9])
    
    # Get the predicted classes for those images.
    cls_pred = cls_pred[incorrect]

    # Get the true classes for those images.
    cls_true = cls_test[incorrect]
    
    # Plot the 9 images we have loaded and their corresponding classes.
    # We have only loaded 9 images so there is no need to slice those again.
    plot_images(images=images,
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

In [50]:
def example_errors():
    # The Keras data-generator for the test-set must be reset
    # before processing. This is because the generator will loop
    # infinitely and keep an internal index into the dataset.
    # So it might start in the middle of the test-set if we do
    # not reset it first. This makes it impossible to match the
    # predicted classes with the input images.
    # If we reset the generator, then it always starts at the
    # beginning so we know exactly which input-images were used.
    generator_test.reset()
    
    # Predict the classes for all images in the test-set.
    y_pred = new_model.predict(generator_test, steps=steps_test)

    # Convert the predicted classes from arrays to integers.
    cls_pred = np.argmax(y_pred,axis=1)

    # Plot examples of mis-classified images.
    plot_example_errors(cls_pred)
    
    # Print the confusion matrix.
    print_confusion_matrix(cls_pred)

In [51]:
def load_images(image_paths):
    # Load the images from disk.
    images = [plt.imread(path) for path in image_paths]

    # Convert to a numpy array and return it.
    return np.asarray(images)

In [52]:
def plot_training_history(history):
    # Get the classification accuracy and loss-value
    # for the training-set.
    acc = history.history['categorical_accuracy']
    loss = history.history['loss']

    # Get it for the validation-set (we only use the test-set).
    val_acc = history.history['val_categorical_accuracy']
    val_loss = history.history['val_loss']

    # Plot the accuracy and loss-values for the training-set.
    plt.plot(acc, linestyle='-', color='b', label='Training Acc.')
    plt.plot(loss, 'o', color='b', label='Training Loss')
    
    # Plot it for the test-set.
    plt.plot(val_acc, linestyle='--', color='r', label='Test Acc.')
    plt.plot(val_loss, 'o', color='r', label='Test Loss')

    # Plot title and legend.
    plt.title('Training and Test Accuracy')
    plt.legend()

    # Ensure the plot shows correctly.
    plt.show()

In [53]:
# dataset_dir = '/content/drive/MyDrive/SleepyWheels/dataset'
# output_dataset_dir = '/content/drive/MyDrive/SleepyWheels/split'

In [54]:
# Train-Val split
# splitfolders.ratio(dataset_dir, output=output_dataset_dir, seed=1337, ratio=(.9, 0.1))

In [55]:
train_dir = '/data/urk18cs085/dataset/train'
test_dir = '/data/urk18cs085/dataset/val'

### Create the model

In [ ]:
import keras_efficientnet_v2
model = keras_efficientnet_v2.EfficientNetV2B0(dropout=1e-6, num_classes=0, pretrained="imagenet")
print(model.output_shape)

In [ ]:
model.summary()

In [ ]:
input_shape = model.layers[0].output_shape[0][1:3]
input_shape

In [67]:
datagen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=180,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=[0.9, 1.5],
      brightness_range=[0.5,1.1],
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

In [68]:
datagen_test = ImageDataGenerator(rescale=1./255)

In [69]:
batch_size = 64

In [ ]:
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

In [ ]:
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

In [72]:
# Check generated images
# x = train_img_gen.next()
# for i in range(0,4):
#     image = x[i]
#     plt.imshow(image)
#     plt.show()

In [ ]:
steps_test = generator_test.n / batch_size
steps_test

In [74]:
image_paths_train = path_join(train_dir, generator_train.filenames)
image_paths_test = path_join(test_dir, generator_test.filenames)

In [75]:
cls_train = generator_train.classes
cls_test = generator_test.classes

In [ ]:
class_names = list(generator_train.class_indices.keys())
class_names

In [ ]:
num_classes = generator_train.num_classes
num_classes

In [ ]:
# Load the first images from the train-set.
images = load_images(image_paths=image_paths_train[0:9])

# Get the true classes for those images.
cls_true = cls_train[0:9]

# Plot the images and labels using our helper-function above.
plot_images(images=images, cls_true=cls_true, smooth=True)

In [79]:
def predict(image_path):
    # Load and resize the image using PIL.
    img = PIL.Image.open(image_path)
    img_resized = img.resize(input_shape, PIL.Image.LANCZOS)

    # Plot the image.
    plt.imshow(img_resized)
    plt.show()

    # Convert the PIL image to a numpy-array with the proper shape.
    img_array = np.expand_dims(np.array(img_resized), axis=0)

    # Use the model to make a prediction.
    # This outputs an array with 2 numbers corresponding to
    # the classes of the dataset.
    pred = model.predict(img_array)
    
    # Decode the output of the model.
    pred_decoded = decode_predictions(pred)[0]

    # Print the predictions.
    for code, name, score in pred_decoded:
        print("{0:>6.2%} : {1}".format(score, name))

In [80]:
# predict(image_path=image_paths_train[0])

In [81]:
# predict(image_path=image_paths_test[0])

### Transfer Learning

In [ ]:
model.summary()

In [34]:
# transfer_layer = model.get_layer('layer_name')

In [35]:
# transfer_layer.output

In [36]:
# conv_model = Model(inputs=model.input,
#                    outputs=transfer_layer.output)

In [37]:
conv_model = model

In [38]:
# Start a new Keras Sequential model.
new_model = Sequential()

# Add the convolutional part of the EffNetV2 model from above.
new_model.add(conv_model)

# Flatten the output of the EffNetV2 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the EffNetV2 model has
# recognized in the image.
new_model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.7))

# Add the final layer for the actual classification.
new_model.add(Dense(num_classes, activation='softmax'))

In [39]:
optimizer = Nadam(learning_rate=3e-3)

In [40]:
loss = 'categorical_crossentropy'

In [41]:
metrics = ['categorical_accuracy']

In [42]:
def print_layer_trainable():
    for layer in conv_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

In [ ]:
print_layer_trainable()

In [44]:
conv_model.trainable = False

for layer in conv_model.layers:
    layer.trainable = False

In [ ]:
print_layer_trainable()

In [46]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [47]:
epochs = 50
# steps_per_epoch = 100

In [ ]:
steps_train = generator_train.n / batch_size
steps_train

In [ ]:
new_model.summary()

In [50]:
checkpoint_path = '/data/urk18cs085/saved_models/transfer_learnt_best_27thFeb.h5'
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

backup_path = '/data/urk18cs085/backup'
backup = BackupAndRestore(backup_path)

lrr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=.01, patience=3, min_lr=1e-4)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

tb = TensorBoard(log_dir='./TransferLearntGraph', histogram_freq=0, write_graph=True, write_images=True)

callbacks = [checkpoint, backup, lrr, earlystop, tb]

In [ ]:
history = new_model.fit(x=generator_train,
                        epochs=epochs,
                        steps_per_epoch=steps_train,
                        callbacks = callbacks,
                        validation_data=generator_test,
                        validation_steps=steps_test)

In [ ]:
plot_training_history(history)

In [ ]:
example_errors()

In [ ]:
!pip uninstall -y tensorboard tb-nightly
!pip install tensorboard

In [ ]:
# %load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir './TransferLearntGraph' --port 8889 --bind_all
# %tensorboard --logdir './TransferLearntGraph/train/events.out.tfevents.1645980487.starship2.7312.0.v2'

In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

### Fine-tuning

In [44]:
from keras.models import load_model
transfer_learnt_filepath = '/data/urk18cs085/saved_models/transfer_learnt_best_27thFeb.h5'
transfer_learnt_model = load_model(transfer_learnt_filepath)

In [ ]:
transfer_learnt_model.evaluate(generator_test)

In [83]:
# conv_model.trainable = True

In [84]:
# for layer in conv_model.layers:
#     # Boolean whether this layer is trainable.
#     selected_layers = ['post_swish', 'post_bn', 'post_conv', 'add_14', 'stack_5_block7_MB_pw_bn', 'stack_5_block7_MB_pw_conv', 'multiply_15']
# #     selected_layers = ['post_swish', 'post_bn', 'post_conv']
#     trainable = (layer.name in selected_layers)
    
#     # Set the layer's bool.
#     layer.trainable = trainable

In [ ]:
for layer in transfer_learnt_model.layers:
    print("{0}:\t{1}".format(layer.trainable, layer.name))

In [ ]:
for layer in transfer_learnt_model.layers:
    layer.trainable = False
    if layer.name == 'EfficientNetV2B0':
        for inner_layer in layer.layers:
            print("{0}:\t{1}".format(inner_layer.trainable, inner_layer.name))
            inner_layer.trainable = False

In [87]:
transfer_learnt_model.trainable = True

selected_before = ['dense_1', 
                   'dropout', 
                   'dense', 
                   'flatten']

selected_for_finetuning = ['post_swish', 
                           'post_bn', 
                           'post_conv', 
                           'add_14', 
                           'stack_5_block7_MB_pw_bn', 
                           'stack_5_block7_MB_pw_conv', 
                           'multiply_15']

for layer in transfer_learnt_model.layers:
    layer.trainable = True
    if layer.name == 'EfficientNetV2B0':
        for inner_layer in layer.layers:
#             if inner_layer.name not in selected_for_finetuning:
                inner_layer.trainable = True

In [ ]:
for layer in transfer_learnt_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

In [ ]:
for layer in transfer_learnt_model.layers:
    if layer.name == 'EfficientNetV2B0':
        for inner_layer in layer.layers:
            print("{0}:\t{1}".format(inner_layer.trainable, inner_layer.name))

In [ ]:
transfer_learnt_model.summary()

In [91]:
optimizer_fine = Nadam(learning_rate=3e-4)

In [ ]:
transfer_learnt_model.compile(optimizer=optimizer_fine, loss=loss, metrics=metrics)

In [93]:
epochs = 50

In [94]:
checkpoint_path = '/data/urk18cs085/saved_models/fine_tuned_best_28Feb.h5'
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

lrr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=.01, patience=3, min_lr=1e-5)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 4,
                          verbose = 1,
                          restore_best_weights = True)

backup_path = '/data/urk18cs085/finetune_backup'
backup = BackupAndRestore(backup_path)

tb = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks = [checkpoint, lrr, earlystop, backup, tb]

In [ ]:
history = transfer_learnt_model.fit(x=generator_train,
                        epochs=epochs,
                        steps_per_epoch=steps_train,
                        callbacks = callbacks,
                        validation_data=generator_test,
                        validation_steps=steps_test)

In [ ]:
plot_training_history(history)

In [ ]:
result = transfer_learnt_model.evaluate(generator_test, steps=steps_test)

In [ ]:
print("Test-set classification accuracy: {0:.2%}".format(result[1]))

In [ ]:
example_errors()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir './Graph' --port 8890 --bind_all